# Syed Hamza Ali
# Task 7: Cartoonify Faces

This task tests your ability to apply Video processing and Face detection techniques to place a cartoon mask instead of your face in the video.

This project includes :

1. Python Notebook (IPYNB)

2. Cartoon mask (JPG/PNG)

3. Video (wav/mp3) OR Screenshot Image (JPG/PNG)

In [4]:
import cv2
import os
import numpy as np

# Face Masker Class Definition

In [5]:
class FaceMasker:
    
    # Constructor method to initialize object
    def __init__(self, mask_path, scale=1.8, recording_path=None):
        
        # Initialize FPS for recording
        self.recording_fps = 60
        
        # Load Haar Cascade classifier for face detection
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
        
        # Initialize video capture from default camera (0)
        self.video_capture = cv2.VideoCapture(0)
        self.scale = scale

        # Load mask image
        self.mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        self.recording_path = recording_path
        
        # Initialize video writer if recording path is provided
        if self.recording_path:
            width, height = int(self.video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(self.video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
            self.video_writer = cv2.VideoWriter(self.recording_path, cv2.VideoWriter_fourcc(*'XVID'), self.recording_fps, (width, height))
        else:
            self.video_writer = None


    # Method to start the face masking process
    def start(self):

        while True:

            ret, frame = self.video_capture.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))
            
            # Iterate through detected faces
            for (x, y, w, h) in faces:

                scaled = int(w * self.scale)
                resized_mask = cv2.resize(self.mask, (scaled, scaled))

                # Calculate coordinates for placing the mask on the face
                x1, y1 = x - (scaled - w) // 2, y - (scaled - h) // 2
                x2, y2 = x1 + scaled, y1 + scaled

                # Check if mask coordinates are within frame boundaries
                if all(0 < p < frame.shape[i] for p, i in zip((x1, x2, y1, y2), (1, 1, 0, 0))):
                    alpha_s = resized_mask[:, :, 3] / 255.0
                    alpha_l = 1.0 - alpha_s
                    for c in range(3):
                        frame[y1:y2, x1:x2, c] = alpha_s * resized_mask[:, :, c] + alpha_l * frame[y1:y2, x1:x2, c]
            
            cv2.imshow('Face Video', frame)   
                 
            # Break loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            if self.video_writer:
                self.video_writer.write(frame)

        # Release video writer if recording was enabled
        if self.video_writer:
            self.video_writer.release()

        # Release video capture and close all windows
        self.video_capture.release()
        cv2.destroyAllWindows()

In [6]:
face_masker = FaceMasker('cartoon_mask.png', 1.8, 'cartoon_mask_video.mp4')
face_masker.start()